Kan module for classification of EEG signals

In [39]:
import torch
import torch.nn as nn
from kan import KAN
import torch.nn.functional as F
from torchinfo import summary

In [55]:
class kanmodule(nn.Module):
    def __init__(self,channels=16):
        super(kanmodule,self).__init__()
        self.kan1 = KAN([529,40,4])
        self.kan2 = KAN([4,40,5])
        self.kan = nn.Sequential(KAN([529,40,5]),KAN([5,40,10]))
        self.net = nn.ModuleList([self.kan for _ in range(channels)])
        self.flatten = nn.Flatten()
        self.nn = self.create_nn()
    def forward(self,x):
        N,C,H,W = x.shape
        xa = torch.squeeze(x)
        list = []
        xa = xa.permute((2,0,1))
        '''
        for _ in range(W):
            int2 = self.kan1(xa[_])
            int2 = self.kan2(int2)
            list.append(int2)
        output = torch.stack(list)
        output = output.permute((1,2,0))
        output = self.flatten(output)'''
        print(xa.shape)
        i = 0
        list = []
        for h in self.net:
            int2 =  h(xa[i])
            i = i +1
            list.append(int2)
        output = torch.stack(list)
        print(output.shape)
        output = output.permute((1,2,0))
        output = self.flatten(output)
        return self.nn(output)
    def create_nn(self): #creates the nn used in the model
        return nn.Sequential(nn.Flatten(),
                             nn.Linear(160,512,bias=True),
                             nn.Dropout(p=0.3),
                             nn.LeakyReLU(),
                             nn.Linear(512,16,bias=True),
                             #nn.Softmax()
        )

In [56]:
model = kanmodule()
summary(model, (40,1,529,16))

torch.Size([16, 40, 529])
torch.Size([16, 40, 10])


Layer (type:depth-idx)                        Output Shape              Param #
kanmodule                                     [40, 16]                  216,800
├─ModuleList: 1-1                             --                        --
│    └─Sequential: 2-1                        [40, 10]                  --
│    │    └─KAN: 3-1                          [40, 5]                   213,600
│    │    └─KAN: 3-2                          [40, 10]                  6,000
│    └─Sequential: 2-2                        [40, 10]                  (recursive)
│    │    └─KAN: 3-3                          [40, 5]                   (recursive)
│    │    └─KAN: 3-4                          [40, 10]                  (recursive)
│    └─Sequential: 2-3                        [40, 10]                  (recursive)
│    │    └─KAN: 3-5                          [40, 5]                   (recursive)
│    │    └─KAN: 3-6                          [40, 10]                  (recursive)
│    └─Sequential: 2-4      

: 